In [3]:
import torch
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import dataloader
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import os
from torchvision import transforms
from PIL import Image
import json

In [40]:
import re
#파일 이름 재정
def remove_trailing_number(filename):
    # 정규식을 사용하여 파일 이름에서 마지막 숫자 부분을 찾음
    match = re.search(r'(_\d+)?\.json$', filename)
    
    if match:
        # 숫자가 발견된 경우, 해당 숫자 부분을 제거
        return filename[:match.start()] + '.json'
    else:
        # 숫자가 없는 경우, 그대로 반환
        return filename
def get_subfolder_names(folder_path):
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    return subfolders
def process_label_file(folder_path,file_name):

    # 라벨 파일 읽기
    file_path = folder_path+"/"+file_name
    with open(file_path, 'r') as file:
        label_data = json.load(file)

    # 'item' 안의 'survey' 부분 가져오기
    survey_data = label_data["item"]["survey"]

    # Q4201부터 Q4216까지의 값이 0이 아니면 1로 변경
    for i in range(4201, 4217):
        key = f"Q{i}"
        survey_data[key] = 1 if survey_data[key] != 0 else 0

    # 'item' 안의 'survey'만을 가진 새로운 라벨 생성
    new_label = {
        "item": {
            "imgName": label_data["item"]["imgName"],
            "era": label_data["item"]["era"],
            "style": label_data["item"]["style"],
            "gender": label_data["item"]["gender"],
            "survey": survey_data
        }
    }

    # 기존 라벨 제거 후 새로운 라벨로 대체
    label_data = new_label["item"]

    new_file_name = remove_trailing_number(file_name)
    new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
    # 새로운 라벨 저장
    with open(new_file_path, 'w') as file:
        json.dump(label_data, file, indent=4)
    if new_file_name == file_name:
        return
    os.remove(file_path)

In [41]:
def label_cleanUp(folder_path):
    folder_file_list = os.listdir(folder_path)
    for file_name in folder_file_list:
        process_label_file(folder_path,file_name)

In [35]:
#test
label_path = 'VL_woman_2019'
label_cleanUp(label_path)

In [42]:
folder_path_training = '../data/Training/label' # 파일 위치에 따라 수정해야함
folder_path_vaildation ='../data/Validation/label' # 파일 위치에 따라 수정해야함
subfolder_names_training = get_subfolder_names(folder_path_training)
subfolder_names_validation = get_subfolder_names(folder_path_vaildation)
print("Training 폴더 안에 있는 모든 서브폴더명:", subfolder_names_training)
print("Vaildation 폴더 안에 있는 모든 서브폴더명:", subfolder_names_validation)

Training 폴더 안에 있는 모든 서브폴더명: ['TL_man_1950', 'TL_man_1960', 'TL_man_1970', 'TL_man_1980', 'TL_man_1990', 'TL_man_2000', 'TL_man_2010', 'TL_man_2019', 'TL_woman_1950', 'TL_woman_1960', 'TL_woman_1970', 'TL_woman_1980', 'TL_woman_1990', 'TL_woman_2000', 'TL_woman_2010', 'TL_woman_2019']
Vaildation 폴더 안에 있는 모든 서브폴더명: ['VL_man_1950', 'VL_man_1960', 'VL_man_1970', 'VL_man_1980', 'VL_man_1990', 'VL_man_2000', 'VL_man_2010', 'VL_man_2019', 'VL_woman_1950', 'VL_woman_1960', 'VL_woman_1970', 'VL_woman_1980', 'VL_woman_1990', 'VL_woman_2000', 'VL_woman_2010', 'VL_woman_2019']


In [44]:
'''
for label_path in subfolder_names_training:
    label_cleanUp(folder_path_training +'/'+ label_path)
    #print(folder_path_training +'/'+ label_path)
#'''
#'''
for label_path in subfolder_names_validation:
    label_cleanUp(folder_path_vaildation +'/'+ label_path)
    #print(folder_path_vaildation+'/'+ label_path)
#'''

In [66]:
def one_hot_group(data,category):
    label_dict = {data: i for i, era in enumerate(category)}
    data_indices = [label_dict[data]]
    one_hot = torch.nn.functional.one_hot(torch.tensor(data_indices), len(category))
    print(one_hot)
    return 
def one_hot_encode_label(folder_path,file_name):
    # 라벨 파일 읽기
    file_path = folder_path+"/"+file_name
    with open(file_path, 'r') as file:
        label_data = json.load(file)

    print(label_data)
    
    era = str(label_data["era"])
    era_category = ['1950','1960','1970','1980','1990','2000','2010','2019']
    print(era)
    
    style = label_data["style"]
    #style_category = []
    
    gender = label_data["gender"]
    gender_category = ['M','W']

    survey_data = label_data["survey"]
    q1 = survey_data["Q1"]
    q42xx_category = ["0","1"]
    '''

    era_label_dict = {era: i for i, era in enumerate(era_category)}
    print(era_label_dict)
    era_indices = [era_label_dict[era]]
    era_one_hot = torch.nn.functional.one_hot(torch.tensor(era_indices), len(era_category))

    '''
    era_one_hot = one_hot_group(era,era_category)
    gender_one_hot = one_hot_group(gender,gender_category)
    
    q42xx_one_hot = []
    for i in range(0, 16):
        key = f"Q{4201+i}"
        q42xx_one_hot.append(torch.nn.functional.one_hot(torch.tensor(survey_data[key]), len(q42xx_category)))
    print(q42xx_one_hot)

def label_one_hot(folder_path):
    folder_file_list = os.listdir(folder_path)
    for file_name in folder_file_list:
        one_hot_encode_label(folder_path,file_name)
label_path = 'VL_woman_2019'
label_one_hot(label_path)

{'imgName': 'T_00306_19_normcore_W.jpg', 'era': 2019, 'style': 'normcore', 'gender': 'W', 'survey': {'Q1': 2, 'Q2': 1, 'Q3': 5, 'Q411': 1, 'Q412': 2, 'Q413': 2, 'Q414': 2, 'Q4201': 0, 'Q4202': 0, 'Q4203': 0, 'Q4204': 0, 'Q4205': 0, 'Q4206': 0, 'Q4207': 0, 'Q4208': 0, 'Q4209': 0, 'Q4210': 1, 'Q4211': 1, 'Q4212': 1, 'Q4213': 1, 'Q4214': 0, 'Q4215': 0, 'Q4216': 0, 'Q5': 1}}
2019
tensor([[0, 0, 0, 0, 0, 0, 0, 1]])
tensor([[0, 1]])
[tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0]), tensor([0, 1]), tensor([0, 1]), tensor([0, 1]), tensor([0, 1]), tensor([1, 0]), tensor([1, 0]), tensor([1, 0])]
{'imgName': 'T_00323_19_normcore_W.jpg', 'era': 2019, 'style': 'normcore', 'gender': 'W', 'survey': {'Q1': 2, 'Q2': 2, 'Q3': 6, 'Q411': 2, 'Q412': 1, 'Q413': 1, 'Q414': 2, 'Q4201': 0, 'Q4202': 0, 'Q4203': 0, 'Q4204': 0, 'Q4205': 0, 'Q4206': 0, 'Q4207': 0, 'Q4208': 0, 'Q4209': 0, 'Q4210': 0, 'Q4211': 1, 'Q4212':

In [22]:
'''
def one_hot_encode_label(labels):

    
    era, gender = labels
    era_category = ['50','60','70','80','90','00','10','19']
    gender_category = ['M','W']
    era_label_dict = {era: i for i, era in enumerate(era_category)}
    era_indices = [era_label_dict[era]]
    era_one_hot = torch.nn.functional.one_hot(torch.tensor(era_indices), len(era_category))

# Gender에 대한 one-hot encoding
    gender_label_dict = {gender: i for i, gender in enumerate(gender_category)}
    gender_indices = [gender_label_dict[gender] ]
    gender_one_hot = torch.nn.functional.one_hot(torch.tensor(gender_indices), len(gender_category))

# Era와 Gender의 one-hot encoding을 연결
    combined_one_hot = torch.cat((era_one_hot, gender_one_hot), dim=1)
    return combined_one_hot
'''